<h1>Assignment 2 - Web Scraping</h1>

**In this assignment you need to work with data from the [worldometers](https://www.worldometers.info/coronavirus/) website. I want you to scrape all 215 countries information about coronavirus cases from the website.**
The data has to include:
- `Country name`
- `Total cases`
- `Total deaths`
- `Total recovered`
- `Active cases`
- `New cases`
- `New deaths`
- `Total tests`
- `Population`

**You need to use beautiful soup 4 and regular expressions for this task. Save results in csv file and read this dataset**

<h3> Import Dependencies
</h3>

In [213]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
from urllib.request import urlopen
from bs4 import BeautifulSoup
import re
import requests

<h3> Starting scrape the data </h3>


In [235]:
url = "https://www.worldometers.info/coronavirus/"
response = requests.get(url)

if response.status_code == 200:
    soup = BeautifulSoup(response.text, 'lxml')
    table = soup.find('table')
    rows = table.find_all('tr')
    list_rows = []

    for row in rows:
        cells = row.find_all('td')
        clean_row = []

        for cell in cells:
            str_cell = str(cell)
            clean = re.compile('<.*?>')
            clean_text = re.sub(clean, '', str_cell)
            clean_text = clean_text.strip()
            clean_text = re.sub(r'\s+', ' ', clean_text)
            clean_row.append(clean_text)
        list_rows.append(clean_row)

    for clean_row in list_rows:
        clean_row

else:
    print("Failed to retrieve the webpage")

df = pd.DataFrame(list_rows)
string_to_check = '<td style="font-weight: bold; text-align:right;">'
df = df[df.apply(lambda row: row.astype(str).str.contains(string_to_check).any(), axis=1)]
df = df.iloc[:, :15]

new_value = ''
df.replace(to_replace=string_to_check, value=new_value, regex=True, inplace=True)

df.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
9,1,USA,"111,790,779",,"1,219,159",,"109,761,610",,"810,010","1,009","333,898","3,641","1,186,851,502","3,544,901","334,805,269"
10,2,India,"45,034,136",,"533,547",,N/A,N/A,N/A,N/A,"32,016",379,"935,879,495","665,334","1,406,631,776"
11,3,France,"40,138,560",,"167,642",,"39,970,918",,0,,"612,013","2,556","271,490,188","4,139,547","65,584,518"
12,4,Germany,"38,827,570",,"182,952",,"38,240,600",,"404,018",N/A,"462,874","2,181","122,332,384","1,458,359","83,883,596"
13,5,Brazil,"38,729,836",,"711,249",,"36,249,161",,"1,769,426",N/A,"179,843","3,303","63,776,166","296,146","215,353,593"


In [236]:
col_str = str(col_labels)
cleantext2 = BeautifulSoup(col_str, "lxml").get_text()
cleantext2 = cleantext2.strip()
cleantext2 = re.sub(r'\s+', ' ', cleantext2)
all_header.append(cleantext2)

df_header = pd.DataFrame(all_header)
df_header = df_header[0].str.split(',', expand=True)
df_header = df_header.iloc[0:1, :17]
df_header.drop(columns=[2], inplace=True)
df_header.drop(columns=[11], inplace=True)
df_header.columns = range(df_header.shape[1])

frames = [df_header, df]
data = pd.concat(frames)
data = data.rename(columns=data.iloc[0])
data.drop(data.columns[0], axis=1, inplace=True)
data.reset_index(drop=True, inplace=True)
data.columns = data.columns.str.replace(' ', '')
data.drop(data.index[0], axis=0, inplace=True)
data.head()

,Country,TotalCases,NewCases,TotalDeaths,NewDeaths,TotalRecovered,NewRecovered,ActiveCases,Serious,TotCases/1Mpop,Deaths/1Mpop,TotalTests,Tests/1Mpop,Population
1,USA,"111,790,779",,"1,219,159",,"109,761,610",,"810,010","1,009","333,898","3,641","1,186,851,502","3,544,901","334,805,269"
2,India,"45,034,136",,"533,547",,N/A,N/A,N/A,N/A,"32,016",379,"935,879,495","665,334","1,406,631,776"
3,France,"40,138,560",,"167,642",,"39,970,918",,0,,"612,013","2,556","271,490,188","4,139,547","65,584,518"
4,Germany,"38,827,570",,"182,952",,"38,240,600",,"404,018",N/A,"462,874","2,181","122,332,384","1,458,359","83,883,596"
5,Brazil,"38,729,836",,"711,249",,"36,249,161",,"1,769,426",N/A,"179,843","3,303","63,776,166","296,146","215,353,593"


In [233]:
# Validation - USA
data[data['Country'] == 'USA']

Index(['Country', 'TotalCases', 'NewCases', 'TotalDeaths', 'NewDeaths',
       'TotalRecovered', 'NewRecovered', 'ActiveCases', 'Serious',
       'TotCases/1Mpop', 'Deaths/1Mpop', 'TotalTests', 'Tests/1Mpop',
       'Population'],
      dtype='object')

<h3> Save & Read CSV
</h3>

In [238]:
#Write File
data.to_csv('coronavirus_data.csv', index=False)

In [239]:
# Read file
new_df = pd.read_csv('coronavirus_data.csv')
new_df.head()

,Country,TotalCases,NewCases,TotalDeaths,NewDeaths,TotalRecovered,NewRecovered,ActiveCases,Serious,TotCases/1Mpop,Deaths/1Mpop,TotalTests,Tests/1Mpop,Population
0,USA,"111,790,779",NaN,"1,219,159",NaN,"109,761,610",NaN,"810,010","1,009","333,898","3,641","1,186,851,502","3,544,901","334,805,269"
1,India,"45,034,136",NaN,"533,547",NaN,NaN,NaN,NaN,NaN,"32,016",379,"935,879,495","665,334","1,406,631,776"
2,France,"40,138,560",NaN,"167,642",NaN,"39,970,918",NaN,0,NaN,"612,013","2,556","271,490,188","4,139,547","65,584,518"
3,Germany,"38,827,570",NaN,"182,952",NaN,"38,240,600",NaN,"404,018",NaN,"462,874","2,181","122,332,384","1,458,359","83,883,596"
4,Brazil,"38,729,836",NaN,"711,249",NaN,"36,249,161",NaN,"1,769,426",NaN,"179,843","3,303","63,776,166","296,146","215,353,593"
